بازیابی متن

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp drive/MyDrive/train.json .


part 0 (data retrival) :

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#Load data from JSON
def load_data(json_file):
    data = []
    with open(json_file, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
    return data


def preprocess_texts(data):
    filtered_data = [item for item in data if item['exp'] is not None and item['question'] is not None]
    texts = [f"{item['question']} {item['exp']}" for item in filtered_data]
    # texts = [f"{item['exp']}" for item in filtered_data]
    return filtered_data, texts

# TF-IDF for texts and user query
def compute_tfidf(texts, query):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts + [query])
    return tfidf_matrix

# Cosine Similarity
def find_similar_texts(tfidf_matrix):
    query_vector = tfidf_matrix[-1]
    similarities = cosine_similarity(query_vector, tfidf_matrix[:-1])
    return similarities.flatten()


def display_related_texts(filtered_data, similarities, top_n=3):
    sorted_indices = np.argsort(similarities)[-top_n:][::-1]
    for idx in sorted_indices:
        print(f"Related text: {filtered_data[idx]['question']} - {filtered_data[idx]['exp']} with similarity {similarities[idx]}")


json_file = 'train.json'
query = "Which vitamin is supplied from only animal source?"

data = load_data(json_file)
filtered_data, texts = preprocess_texts(data)
tfidf_matrix = compute_tfidf(texts, query)
similarities = find_similar_texts(tfidf_matrix)
display_related_texts(filtered_data, similarities)


Related text: Which vitamin is supplied from only animal source: - Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. P 640* Vitamin B12 (Cobalamin) is synthesized solely by microorganisms.* In humans, the only source for humans is food of animal origin, e.g., meat, fish, and dairy products.* Vegetables, fruits, and other foods of nonanimal origin doesn't contain Vitamin B12 .* Daily requirements of vitamin Bp is about 1-3 pg. Body stores are of the order of 2-3 mg, sufficient for 3-4 years if supplies are completely cut off. with similarity 0.4810651098426298
Related text: Which of the following is the major source of Vitamin D - Vitamin D is not strictly a vitamin since it can be synthesized in the skin, and under most conditions, this is the major source of the Vitamin D. Ref : Biochemistry by U. Satyanarayana 3rd edition Pgno : 123 with similarity 0.4054725979423891
Related text: Source of vitamin B12? - Ans. (a) Animal SourceRef: Harrison / 640Only source of vitamin B12 for humans is f

ساخت پاسخ کوتاه برای دیتاست سوال و جواب

In [3]:
%pip install transformers

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer


def load_data(json_file):
    data = []
    with open(json_file, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
    return data

def preprocess_texts(data):
    filtered_data = [item for item in data if item['exp'] is not None and item['question'] is not None]
    texts = [f"{item['question']} {item['exp']}" for item in filtered_data]
    return filtered_data, texts

def compute_tfidf(texts, query):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts + [query])
    return tfidf_matrix

def find_similar_texts(tfidf_matrix):
    query_vector = tfidf_matrix[-1]
    similarities = cosine_similarity(query_vector, tfidf_matrix[:-1])
    return similarities.flatten()

def get_related_texts(filtered_data, similarities, top_n=1):
    sorted_indices = np.argsort(similarities)[-top_n:][::-1]
    related_texts = [filtered_data[idx]['exp'] for idx in sorted_indices]
    print("related texts : ",related_texts)
    return related_texts

def create_input(query, related_texts):
    input_text = f"question: {query} related_text: {' '.join(related_texts)}"
    return input_text

def generate_answer(model, tokenizer, input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(input_ids, max_length=15000, num_beams=15, early_stopping=True , temperature=0.1, top_k=None, top_p=None)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

json_file = 'train.json'
query = "Which vitamin is supplied from only animal source?"

data = load_data(json_file)
filtered_data, texts = preprocess_texts(data)

tfidf_matrix = compute_tfidf(texts, query)
similarities = find_similar_texts(tfidf_matrix)
related_texts = get_related_texts(filtered_data, similarities)

model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

input_text = create_input(query, related_texts)
print(f"Input text for the model: {input_text}")

answer = generate_answer(model, tokenizer, input_text)
print(f"Generated answer: {answer}")


related texts :  ["Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. P 640* Vitamin B12 (Cobalamin) is synthesized solely by microorganisms.* In humans, the only source for humans is food of animal origin, e.g., meat, fish, and dairy products.* Vegetables, fruits, and other foods of nonanimal origin doesn't contain Vitamin B12 .* Daily requirements of vitamin Bp is about 1-3 pg. Body stores are of the order of 2-3 mg, sufficient for 3-4 years if supplies are completely cut off."]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Input text for the model: question: Which vitamin is supplied from only animal source? related_text: Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. P 640* Vitamin B12 (Cobalamin) is synthesized solely by microorganisms.* In humans, the only source for humans is food of animal origin, e.g., meat, fish, and dairy products.* Vegetables, fruits, and other foods of nonanimal origin doesn't contain Vitamin B12 .* Daily requirements of vitamin Bp is about 1-3 pg. Body stores are of the order of 2-3 mg, sufficient for 3-4 years if supplies are completely cut off.


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated answer: B12


ساخت پاسخ بلند برای دیتاست سوال و جواب

part 1 :using llm to generate apropriate answer from retrived data

In [ ]:

import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer

def load_data(json_file):
    data = []
    with open(json_file, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
    return data

def preprocess_texts(data):
    filtered_data = [item for item in data if item['exp'] is not None and item['question'] is not None]
    texts = [f"{item['question']} {item['exp']}" for item in filtered_data]
    return filtered_data, texts

def compute_tfidf(texts, query):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts + [query])
    return tfidf_matrix

def find_similar_texts(tfidf_matrix):
    query_vector = tfidf_matrix[-1]
    similarities = cosine_similarity(query_vector, tfidf_matrix[:-1])
    return similarities.flatten()

def get_related_texts(filtered_data, similarities, top_n=3):
    sorted_indices = np.argsort(similarities)[-top_n:][::-1]
    related_texts = [filtered_data[idx]['exp'] for idx in sorted_indices]
    return related_texts

def create_input(query, related_texts):
    input_text = f"پرسش: {query} زمینه: {' '.join(related_texts)}"
    return input_text

# Generate answer using the model
def generate_answer(model, tokenizer, input_text, max_length=150, num_beams=5, early_stopping=True, temperature=1.0, top_k=None, top_p=None, repetition_penalty=1.0, no_repeat_ngram_size=2, length_penalty=2.0, do_sample=True):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=early_stopping,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        length_penalty=length_penalty,
        do_sample=do_sample
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


json_file = 'train.json'
query = "Which vitamin is supplied from only animal source?"

data = load_data(json_file)
filtered_data, texts = preprocess_texts(data)

tfidf_matrix = compute_tfidf(texts, query)
similarities = find_similar_texts(tfidf_matrix)
related_texts = get_related_texts(filtered_data, similarities)

model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

input_text = create_input(query, related_texts)
print(f"Input text for the model: {input_text}")

answer = generate_answer(
    model, tokenizer, input_text,
    max_length=150,
    num_beams=10,
    early_stopping=False,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=2.0,
    no_repeat_ngram_size=3,
    length_penalty=3.0,
    do_sample=True
)
print(f"Generated answer: {answer}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Input text for the model: پرسش: Which vitamin is supplied from only animal source? زمینه: Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. P 640* Vitamin B12 (Cobalamin) is synthesized solely by microorganisms.* In humans, the only source for humans is food of animal origin, e.g., meat, fish, and dairy products.* Vegetables, fruits, and other foods of nonanimal origin doesn't contain Vitamin B12 .* Daily requirements of vitamin Bp is about 1-3 pg. Body stores are of the order of 2-3 mg, sufficient for 3-4 years if supplies are completely cut off. Vitamin D is not strictly a vitamin since it can be synthesized in the skin, and under most conditions, this is the major source of the Vitamin D. Ref : Biochemistry by U. Satyanarayana 3rd edition Pgno : 123 Ans. (a) Animal SourceRef: Harrison / 640Only source of vitamin B12 for humans is food of animal origin, e.g., meat, fish, and dairy products.Vegetables, fruits, and other foods of non-animal origin are free from cobalamin unless they are co

ذخیره سازی چیکده مقالات

In [4]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requir

In [5]:
from datasets import load_dataset
import os
import json

# Load all in streaming mode
full_pubmed = load_dataset('pubmed', streaming=True)

if not os.path.isdir('pubmed_portions'):
    os.mkdir('pubmed_portions')

portion_size = 1000000  # 1 million entries per portion
pubmed_portion = []
counter = 0

for idx, entry in enumerate(full_pubmed['train']):
    try:
        # Check necessary fields
        pmid = entry.get('MedlineCitation', {}).get('PMID', None)
        year = entry.get('MedlineCitation', {}).get('DateCompleted', {}).get('Year', None)
        abstract_text = entry.get('MedlineCitation', {}).get('Article', {}).get('Abstract', {}).get('AbstractText', None)
        abstract_title = entry.get('MedlineCitation', {}).get('Article', {}).get('ArticleTitle', None)
        author_list = entry.get('MedlineCitation', {}).get('Article', {}).get('AuthorList', {}).get('Author', [])

        if isinstance(author_list, dict):
            author_list = [author_list]
        abstract_authors_list = [author.get('LastName', '') for author in author_list if 'LastName' in author]

        if pmid and year and abstract_text and abstract_title and abstract_authors_list:
            if len(abstract_text) > 20 and len(abstract_title) > 5 and len(abstract_authors_list) > 0:
                pubmed_portion.append({
                    'pmid': pmid,
                    'year': year,
                    'abstract_text': abstract_text,
                    'abstract_title': abstract_title,
                    'abstract_authors_list': abstract_authors_list
                })

        if idx % 10000 == 0:
            print(f'Processed {idx} entries')


        if len(pubmed_portion) >= portion_size:
            with open(f'pubmed_portions/pubmed_portion_{counter}.json', 'w') as fp:
                json.dump(pubmed_portion, fp, ensure_ascii=False, indent=4)
            pubmed_portion = []
            counter += 1

        if idx == 50000:
            print(f'{idx} entries processed, stopping.')
            break

    except Exception as e:
        print(f'Error processing entry {idx}: {e}')
        pass

if len(pubmed_portion) > 0:
    with open(f'pubmed_portions/pubmed_portion_{counter}.json', 'w') as fp:
        json.dump(pubmed_portion, fp, ensure_ascii=False, indent=4)
    print(f'Saved remaining entries to pubmed_portion_{counter}.json')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for pubmed contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/pubmed.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
Processed 0 entries
Processed 10000 entries
Processed 20000 entries
Processed 30000 entries
Processed 40000 entries
Processed 50000 entries
50000 entries processed, stopping.
Saved remaining entries to pubmed_portion_0.json


ساخت پاسخ برای دیتاست چکیده مقالات

In [6]:
%cp pubmed_portions/pubmed_portion_0.json .

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer

def load_data(json_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

def preprocess_texts(data):
    filtered_data = [item for item in data if item['abstract_text'] is not None and item['abstract_title'] is not None]
    texts = [f"{item['abstract_title']} {item['abstract_text']}" for item in filtered_data]
    return filtered_data, texts

def compute_tfidf(texts, query):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts + [query])
    return tfidf_matrix

def find_similar_texts(tfidf_matrix):
    query_vector = tfidf_matrix[-1]
    similarities = cosine_similarity(query_vector, tfidf_matrix[:-1])
    return similarities.flatten()

def get_related_texts(filtered_data, similarities, top_n=3):
    sorted_indices = np.argsort(similarities)[-top_n:][::-1]
    related_texts = [filtered_data[idx]['abstract_text'] for idx in sorted_indices]
    return related_texts

def create_input(query, related_texts):
    input_text = f"پرسش: {query} زمینه: {' '.join(related_texts)}"
    return input_text

def generate_answer(model, tokenizer, input_text, max_length=150, num_beams=5, early_stopping=True, temperature=1.0, top_k=None, top_p=None, repetition_penalty=1.0, no_repeat_ngram_size=2, length_penalty=2.0, do_sample=True):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=early_stopping,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        length_penalty=length_penalty,
        do_sample=do_sample
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

json_file = 'pubmed_portion_0.json'
query = "Which vitamin is supplied from only animal source?"

data = load_data(json_file)
filtered_data, texts = preprocess_texts(data)

tfidf_matrix = compute_tfidf(texts, query)
similarities = find_similar_texts(tfidf_matrix)
related_texts = get_related_texts(filtered_data, similarities)

model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

input_text = create_input(query, related_texts)
print(f"Input text for the model: {input_text}")

answer = generate_answer(
    model, tokenizer, input_text,
    max_length=150,
    num_beams=10,
    early_stopping=False,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=2.0,
    no_repeat_ngram_size=3,
    length_penalty=3.0,
    do_sample=True
)
print(f"Generated answer: {answer}")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Token indices sequence length is longer than the specified maximum sequence length for this model (732 > 512). Running this sequence through the model will result in indexing errors


Input text for the model: پرسش: Which vitamin is supplied from only animal source? زمینه: Reports concerning the interaction between steroidal contraceptives (the combined pill) and vitamins indicate that in users the mean serum-vitamin-A level is raised and the mean serum-vitamin-B2 (riboflavine), vitamin-B6 (pyridoxine), vitamine-C, folic-acid, and vitamin-B12 levels are reduced. Other vitamins have been insufficiently studied for comment. Biochemical evidence of co-enzyme deficiency has been reported for vitamin B2, vitamin B6, and folic acid. Clinical effects due to vitamin deficiency have been described for vitamin B6--namely, depression and impaired glucose tolerance. Folic-acid deficiency with megaloblastic anaemia has been reported in only 21 cases. The binding of vitamin B-12 to serum proteins during heating at alkaline pH was investigated by gel filtration of serum supplemented with cyano[57Co]-cobalamin. Heating for 5 min at 100 degrees C destroyed most of the vitamin B-12 b

In [ ]:
!cp drive/MyDrive/train.json .

فاین تیون کردن مدل

part 3 ( fine tunning llm by train.json) :

In [12]:
import json
import torch
from datasets import Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

def load_finetune_data(json_file,limit = 2000):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = [json.loads(line) for line in file][:limit]
    return data

def preprocess_finetune_data(data):
    inputs = [f"پرسش: {item['question']}" for item in data]
    targets = [item['exp'] for item in data]
    return inputs, targets

def tokenize_data(tokenizer, inputs, targets, max_length=512):
    encodings = tokenizer(list(map(str, inputs)), truncation=True, padding=True, max_length=max_length)
    labels = tokenizer(list(map(str, targets)), truncation=True, padding=True, max_length=max_length).input_ids

    # Replace padding token by -100
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels]

    encodings['labels'] = labels
    return encodings

def finetune_model(model, tokenizer, train_dataset, eval_dataset, output_dir="./results"):
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=6,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=1000,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset
    )

    trainer.train()

json_file = 'train.json'
finetune_data = load_finetune_data(json_file,limit=2000)
inputs, targets = preprocess_finetune_data(finetune_data)

train_inputs, eval_inputs, train_targets, eval_targets = train_test_split(inputs, targets, test_size=0.1)

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

train_encodings = tokenize_data(tokenizer, train_inputs, train_targets)
eval_encodings = tokenize_data(tokenizer, eval_inputs, eval_targets)

train_dataset = Dataset.from_dict(train_encodings)
eval_dataset = Dataset.from_dict(eval_encodings)

finetune_model(model, tokenizer, train_dataset, eval_dataset)

model.save_pretrained("./finetuned_model_6e")
tokenizer.save_pretrained("./finetuned_model_6e")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,5.182100,4.716797
2,4.783800,4.364937
3,4.507000,4.234683
4,4.380300,4.154284
5,4.342100,4.093639
6,4.361300,4.076693


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


('./finetuned_model_6e/tokenizer_config.json',
 './finetuned_model_6e/special_tokens_map.json',
 './finetuned_model_6e/spiece.model',
 './finetuned_model_6e/added_tokens.json')

ساخت پرامپت با مدل زیانی فاین تیون شده

part 4 (rag using fine tuned llm : )

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer

def load_data(json_file):
    data = []
    with open(json_file, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
    return data

def preprocess_texts(data):
    filtered_data = [item for item in data if item['exp'] is not None and item['question'] is not None]
    texts = [f"{item['question']} {item['exp']}" for item in filtered_data]
    return filtered_data, texts

def compute_tfidf(texts, query):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts + [query])
    return tfidf_matrix

def find_similar_texts(tfidf_matrix):
    query_vector = tfidf_matrix[-1]
    similarities = cosine_similarity(query_vector, tfidf_matrix[:-1])
    return similarities.flatten()

def get_related_texts(filtered_data, similarities, top_n=3):
    sorted_indices = np.argsort(similarities)[-top_n:][::-1]
    related_texts = [filtered_data[idx]['exp'] for idx in sorted_indices]
    return related_texts

def create_input(query, related_texts):
    input_text = f"پرسش: {query} زمینه: {' '.join(related_texts)}"
    return input_text

def generate_answer(model, tokenizer, input_text, max_length=150, num_beams=5, early_stopping=True, temperature=1.0, top_k=None, top_p=None, repetition_penalty=1.0, no_repeat_ngram_size=2, length_penalty=2.0, do_sample=True):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=early_stopping,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        length_penalty=length_penalty,
        do_sample=do_sample
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

json_file = 'train.json'
query = "Which vitamin is supplied from only animal source?"

data = load_data(json_file)
filtered_data, texts = preprocess_texts(data)

tfidf_matrix = compute_tfidf(texts, query)
similarities = find_similar_texts(tfidf_matrix)
related_texts = get_related_texts(filtered_data, similarities)

model_path = "finetuned_model/"  # Adjust this to your actual path
model = T5ForConditionalGeneration.from_pretrained(model_path, local_files_only=True)
tokenizer = T5Tokenizer.from_pretrained(model_path, local_files_only=True)

input_text = create_input(query, related_texts)
print(f"Input text for the model: {input_text}")

answer = generate_answer(
    model, tokenizer, input_text,
    max_length=150,
    num_beams=10,
    early_stopping=False,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=2.0,
    no_repeat_ngram_size=3,
    length_penalty=3.0,
    do_sample=True
)
print(f"Generated answer: {answer}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Input text for the model: پرسش: Which vitamin is supplied from only animal source? زمینه: Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. P 640* Vitamin B12 (Cobalamin) is synthesized solely by microorganisms.* In humans, the only source for humans is food of animal origin, e.g., meat, fish, and dairy products.* Vegetables, fruits, and other foods of nonanimal origin doesn't contain Vitamin B12 .* Daily requirements of vitamin Bp is about 1-3 pg. Body stores are of the order of 2-3 mg, sufficient for 3-4 years if supplies are completely cut off. Vitamin D is not strictly a vitamin since it can be synthesized in the skin, and under most conditions, this is the major source of the Vitamin D. Ref : Biochemistry by U. Satyanarayana 3rd edition Pgno : 123 Ans. (a) Animal SourceRef: Harrison / 640Only source of vitamin B12 for humans is food of animal origin, e.g., meat, fish, and dairy products.Vegetables, fruits, and other foods of non-animal origin are free from cobalamin unless they are co

مقایسه و ارزیابی

part 5 (evaluation) :

In [3]:
!pip install transformers datasets sacrebleu rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a1cc21a34cbca33494d92416b7ab253f76ec9c37ed1146a8eed09da7d1a6946e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [5]:
import json


input_file = 'train.json'
output_file = 'evaluation_data.json'

def convert_data(input_file, output_file):
    data = []

    with open(input_file, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                item = json.loads(line.strip())
                # استخراج فیلدهای مورد نظر
                new_item = {
                    "question": item["question"],
                    "exp": item["exp"]
                }
                data.append(new_item)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")

    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

convert_data(input_file, output_file)
print(f"Data converted and saved to {output_file}")


KeyError: 'exp'

In [9]:
import json

input_file = 'train.json'
output_file = 'evaluation_data.json'

def convert_data(input_file, output_file, start_line=10000, end_line=10050):
    data = []

    with open(input_file, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if i < start_line:
                continue
            if i > end_line:
                break
            try:
                item = json.loads(line.strip())
                new_item = {
                    "question": item["question"],
                    "exp": item["exp"]
                }
                data.append(new_item)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON at line {i}: {e}")

    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

convert_data(input_file, output_file)
print(f"Data converted and saved to {output_file}")


Data converted and saved to evaluation_data.json


In [14]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_metric

def load_data(json_file, limit=None):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    if limit:
        data = data[:limit]
    return data

def preprocess_texts(data):
    filtered_data = [item for item in data if item['exp'] is not None and item['question'] is not None]
    texts = [f"{item['question']} {item['exp']}" for item in filtered_data]
    return filtered_data, texts

def compute_tfidf(texts, query):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts + [query])
    return tfidf_matrix

def find_similar_texts(tfidf_matrix):
    query_vector = tfidf_matrix[-1]
    similarities = cosine_similarity(query_vector, tfidf_matrix[:-1])
    return similarities.flatten()

def get_related_texts(filtered_data, similarities, top_n=3):
    sorted_indices = np.argsort(similarities)[-top_n:][::-1]
    related_texts = [filtered_data[idx]['exp'] for idx in sorted_indices]
    return related_texts

def create_input(query, related_texts):
    input_text = f"پرسش: {query} زمینه: {' '.join(related_texts)}"
    return input_text

def generate_answer(model, tokenizer, input_text, max_length=150, num_beams=5, early_stopping=True, temperature=1.0, top_k=None, top_p=None, repetition_penalty=1.0, no_repeat_ngram_size=2, length_penalty=2.0, do_sample=True):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=early_stopping,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        length_penalty=length_penalty,
        do_sample=do_sample
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

eval_file = 'evaluation_data.json'
eval_data = load_data(eval_file, limit=50)

bleu = load_metric("sacrebleu")
# rouge = load_metric("rouge")
# meteor = load_metric("meteor")

base_model_path = "t5-small"
base_model = T5ForConditionalGeneration.from_pretrained(base_model_path)
base_tokenizer = T5Tokenizer.from_pretrained(base_model_path)

finetuned_model_path = "finetuned_model_6e/"
finetuned_model = T5ForConditionalGeneration.from_pretrained(finetuned_model_path, local_files_only=True)
finetuned_tokenizer = T5Tokenizer.from_pretrained(finetuned_model_path, local_files_only=True)

def evaluate_model(model, tokenizer, eval_data):
    references = []
    predictions = []
    for idx, item in enumerate(eval_data):
        query = item['question']
        reference = item['exp']
        _, texts = preprocess_texts([item])
        if not texts:
            print(f"Skipping item {idx} with empty texts: {item}")
            continue
        tfidf_matrix = compute_tfidf(texts, query)
        if tfidf_matrix.shape[0] == 0:
            print(f"Empty TF-IDF matrix for query {idx}: {query}")
            continue
        similarities = find_similar_texts(tfidf_matrix)
        if len(similarities) == 0:
            print(f"No similarities found for query {idx}: {query}")
            continue
        related_texts = get_related_texts([item], similarities)
        input_text = create_input(query, related_texts)
        prediction = generate_answer(model, tokenizer, input_text)
        references.append(reference)
        predictions.append(prediction)
        print(f"Processed item {idx}")
    return references, predictions

print("Evaluating base model...")
base_references, base_predictions = evaluate_model(base_model, base_tokenizer, eval_data)

print("Evaluating fine-tuned model...")
finetuned_references, finetuned_predictions = evaluate_model(finetuned_model, finetuned_tokenizer, eval_data)

base_bleu_score = bleu.compute(predictions=base_predictions, references=[[ref] for ref in base_references])
# base_rouge_score = rouge.compute(predictions=base_predictions, references=base_references)
# base_meteor_score = meteor.compute(predictions=base_predictions, references=base_references)

finetuned_bleu_score = bleu.compute(predictions=finetuned_predictions, references=[[ref] for ref in finetuned_references])
# finetuned_rouge_score = rouge.compute(predictions=finetuned_predictions, references=finetuned_references)
# finetuned_meteor_score = meteor.compute(predictions=finetuned_predictions, references=finetuned_references)

print("Base Model BLEU Score:", base_bleu_score)
# print("Base Model ROUGE Score:", base_rouge_score)
# print("Base Model METEOR Score:", base_meteor_score)

print("Fine-tuned Model BLEU Score:", finetuned_bleu_score)
# print("Fine-tuned Model ROUGE Score:", finetuned_rouge_score)
# print("Fine-tuned Model METEOR Score:", finetuned_meteor_score)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Evaluating base model...
Skipping item 0 with empty texts: {'question': 'Retinoscopy in 5 year old is best done with:', 'exp': None}
Processed item 1
Processed item 2
Processed item 3
Processed item 4
Processed item 5
Skipping item 6 with empty texts: {'question': 'True about streptococcus:', 'exp': None}


Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors


Processed item 7
Processed item 8
Processed item 9
Processed item 10
Processed item 11
Skipping item 12 with empty texts: {'question': 'Patient diagnosed to have malaria, smear shows all stages of schizonts 14-20 merozoites, yellowish - brown pigment. The type of malaria is -', 'exp': None}
Processed item 13
Processed item 14
Processed item 15
Skipping item 16 with empty texts: {'question': 'Crude birth rate denominator is -', 'exp': None}
Processed item 17
Processed item 18
Processed item 19
Skipping item 20 with empty texts: {'question': 'Intrauterine exposure of diethylstilboestrol is associated with -', 'exp': None}
Processed item 21
Processed item 22
Processed item 23
Processed item 24
Processed item 25
Processed item 26
Skipping item 27 with empty texts: {'question': 'Which of the following Antiepileptic acts by opening Potassium channel?', 'exp': None}
Processed item 28
Processed item 29
Processed item 30
Processed item 31
Processed item 32
Processed item 33
Processed item 34
Pr

Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors


Processed item 7
Processed item 8
Processed item 9
Processed item 10
Processed item 11
Skipping item 12 with empty texts: {'question': 'Patient diagnosed to have malaria, smear shows all stages of schizonts 14-20 merozoites, yellowish - brown pigment. The type of malaria is -', 'exp': None}
Processed item 13
Processed item 14
Processed item 15
Skipping item 16 with empty texts: {'question': 'Crude birth rate denominator is -', 'exp': None}
Processed item 17
Processed item 18
Processed item 19
Skipping item 20 with empty texts: {'question': 'Intrauterine exposure of diethylstilboestrol is associated with -', 'exp': None}
Processed item 21
Processed item 22
Processed item 23
Processed item 24
Processed item 25
Processed item 26
Skipping item 27 with empty texts: {'question': 'Which of the following Antiepileptic acts by opening Potassium channel?', 'exp': None}
Processed item 28
Processed item 29
Processed item 30
Processed item 31
Processed item 32
Processed item 33
Processed item 34
Pr